In [18]:
import os
import json
import requests
import asyncio
from pydantic import BaseModel, Field
from typing import Optional

In [23]:


class ExamQuestion(BaseModel):
    '''考试题目信息'''
    question: str = Field(description="考试题目")
    answer: str = Field(description="考试题目答案")
    question_type: str = Field(description="考试题目类型")
    knowledge_point: str = Field(description="考试题目所属的知识点")
    knowledge_point_description: str = Field(description="考试题目所属的知识点的具体描述")
    extra_requirement: Optional[str] = Field(description="考试题目额外要求")

exam_question = ExamQuestion(
    question='''
    搜索算法相关\n（1）分别说明 DFS 和 BFS 如何用队列或栈实现，并对比两者遍历同一图时的顺序差异。\n（2）在求解无权图最短路径问题时，为什么 BFS 通常比 DFS 更高效？结合遍历特性解释原因。
    ''',
    answer="（1）DFS 用栈（递归或显式栈），一路深入再回溯；BFS 用队列，一层层扩展；顺序差异：DFS 纵深，BFS 横扩。\n（2）BFS 按层扩展，首次到达目标即最短路径；DFS 可能深入很长非最短路径才回溯，访问节点更多。",
    question_type="简答题",
    knowledge_point="",
    knowledge_point_description="",
    extra_requirement="将简答题修改为单选题，然后再核查考题",
)

text = '''
<question info>
question:{question}
answer:{answer}
question_type:{question_type}
knowledge_point:{knowledge_point}
knowledge_point_description:{knowledge_point_description}
extra_requirement:{extra_requirement}
</question info>
'''
inputs = text.format(**exam_question.model_dump())

def test_deployed_agent():
    # 准备测试负载
    payload = {
        "input": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "方案B"},
                ],
            },
        ],
        "session_id": "test_session_002",
        "user_id": "test_user_002",
    }

    print("🧪 测试部署的智能体...")

    # 测试流式响应
    try:
        response = requests.post(
            "http://192.168.60.204:5000/process",
            json=payload,
            stream=True,
            timeout=30,
        )
        res = []
        # 逐行解析 SSE 数据并即时打印
        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue

            # 仅处理 data: 开头的行
            if line.startswith("data:"):
                s = line.split(":", 1)[1].strip()
                if s == "[DONE]":
                    break

                # 尝试解析 JSON；失败时直接原样打印
                try:
                    data = json.loads(s)
                except json.JSONDecodeError:
                    print(s, end="", flush=True)
                    continue

                obj = data.get("object")
                if obj == "content":
                    text = data.get("text", "")
                    if text:
                        print(text, end="", flush=True)
                        res.append(text)
                elif obj == "status":
                    st = data.get("status") or data.get("text") or ""
                    if st:
                        print(f"\n[状态] {st}\n", end="", flush=True)
                elif obj == "error":
                    print(f"\n[错误] {data.get('text') or data}\n", flush=True)
                    break
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 流式测试失败: {e}")
    except requests.exceptions.RequestException as e:
        print(f"ℹ️ JSON端点不可用或失败: {e}")


# Run the test
test_deployed_agent()

🧪 测试部署的智能体...
好的，我选择方案B：先对原简答题进行核查修正，确保内容质量。让我先调用核查工具来检查您提供的简答题。现在让我基于核查建议对考题进行修正：## 核查修正完成

我已经完成了对原简答题的核查和修正。核查结果显示：

**核查结果**：✅ 考题设计良好，符合简答题要求

**修正建议**：
- 对答案（1）部分进行了完善说明
- 答案（2）部分内容正确，保持原样

**修正后的考题内容**：
- **题目**：保持不变
- **答案**：已根据建议进行了完善

现在您有了经过核查修正的高质量简答题内容，可以基于这个内容来设计单选题了。您可以：
1. 将两个子问题合并为一个单选题题干
2. 设计4个合理的选项（A、B、C、D）
3. 确定正确答案

如果您需要我协助设计单选题的选项，请告诉我，我可以基于修正后的内容为您提供建议。